<a href="https://colab.research.google.com/github/keynes99/MetNumUN2024II/blob/main/LAB_ATQ_MACD_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   Q18 Machine Learning Rolling Basis

In this example we predict whether the price will rise or fall by using supervised learning (Bayesian Ridge Regression). This template represents a starting point for developing a system which can take part to the **Q18 NASDAQ-100 Stock Long-Short contest**.

It consists of two parts.

* In the **first part** we just perform a global training of the time series using all time series data. We disregard the sequential aspect of the data and use also future data to train past data.

* In the **second part** we use the built-in backtester and perform training and prediction on a rolling basis in order to avoid forward looking. Please note that we are using a **specialized** version of the Quantiacs backtester which dramatically speeds up the the backtesting process by retraining your model on a regular basis.

**Features for learning**: we will use several technical indicators trying to capture different features. You can have a look at [**Technical Indicators**](https://quantiacs.com/documentation/en/user_guide/technical_indicators.html).

Please note that:

* Your trading algorithm can open short and long positions.

* At each point in time your algorithm can trade all or a subset of the stocks which at that point of time are or were part of the NASDAQ-100 stock index. Note that the composition of this set changes in time, and Quantiacs provides you with an appropriate filter function for selecting them.

* The Sharpe ratio of your system since January 1st, 2006, has to be larger than 1.

* Your system cannot be a copy of the current examples. We run a correlation filter on the submissions and detect duplicates.

* For simplicity we will use a single asset. It pays off to use more assets, ideally uncorrelated, and diversify your positions for a more solid Sharpe ratio.

More details on the rules can be found [here](https://quantiacs.com/contest).

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.

**More help with Jupyter?** Check the official [**Jupyter**](https://jupyter.org/) page.

Once you are done, click on **Submit to the contest** and take part to our competitions.

API reference:

* **data**: check how to work with [data](https://quantiacs.com/documentation/en/reference/data_load_functions.html);

* **backtesting**: read how to run the [simulation](https://quantiacs.com/documentation/en/reference/evaluation.html) and check the results.

Need to use the optimizer function to automate tedious tasks?

* **optimization**: read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).

# How to submit a strategy to the competition?

Check that weights are fine:

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

If everything is ok, write the weights to file:

```python
qnout.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the **Submit** button;
* select the type of competition.

At the beginning you will find the strategy under the **Checking** area:

* **Sent strategies** > **Checking**.

If technical checks are successful, the strategy will go under the **Candidates** area:

* **Sent strategies** > **Candidates**.

Otherwise it will be **Filtered**:

* **Sent strategies** > **Filtered**

and you should inspect error and warning messages.

Note that a strategy under the **Candidates** area should have a Sharpe ratio larger than 1 for being eligible for a prize. Please check warning messages in your **Candidates** area!

**MACD strategy is based on the Moving Average Convergence/Divergence (MACD) indicator. It enters long whenever MACD Histogram changes from negative to positive, and switches to short if the Histogram changes from positive to negative.**

In [ ]:
! pip install git+https://github.com/quantiacs/toolbox.git 2>/dev/null

In [3]:
import xarray as xr
import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout
import qnt.stats as qnstats
import qnt.graph as qngraph

# MACD function
def macd(close, fast_period=12, slow_period=26, signal_period=9):
    macd_line = qnta.ema(close, fast_period) - qnta.ema(close, slow_period)
    macd_signal = qnta.ema(macd_line, signal_period)
    return macd_line, macd_signal

# Improved strategy with MACD and stronger momentum approach
def strategy(data):
    close = data.sel(field="close")
    ma_slow = qnta.lwma(close, 50)
    ma_fast = qnta.lwma(close, 10)

    # Calculate MACD
    macd_line, macd_signal = macd(close)

    # Strong momentum strategy
    # Long when MACD crosses above signal line and fast MA > slow MA
    signals = xr.where((macd_line > macd_signal) & (ma_fast > ma_slow), 1, 1)

    # Optionally add short logic or stop-loss/take-profit filters here
    return signals

# Load data (use more assets for diversification if possible)
data = qndata.stocks.load_ndx_data(min_date="2005-01-01")  # Or use different sets for better diversification

# Apply strategy to the data
output = strategy(data)

# Clean output and make sure it's error-free
output = qnout.clean(output, data)

# Check the output
qnout.check(output, data)

# Save the results (for competition submission)
qnout.write(output)

# Calculate stats
stats = qnstats.calc_stat(data, output.sel(time=slice("2006-01-01", None)))

# Display the stats
print(stats.to_pandas().tail())

# Generate performance plot
qngraph.make_major_plots(stats)



ModuleNotFoundError: No module named 'qnt'